# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentiation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where {T}
    lattice=T(a) * I(3)  # lattice is a cube of $a$ Bohrs
    # Helium at the center of the box
    atoms     = [ElementPsp(:He, psp=load_psp("hgh/lda/He-q2"))]
    positions = [[1/2, 1/2, 1/2]]

    model = model_DFT(lattice, atoms, positions, [:lda_x, :lda_c_vwn];
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...); tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770929695973                   -0.53    9.0         
  2   -2.772149069712       -2.91       -1.32    1.0    201ms
  3   -2.772170390287       -4.67       -2.49    1.0    166ms
  4   -2.772170663647       -6.56       -3.24    1.0    168ms
  5   -2.772170721890       -7.23       -3.93    2.0    192ms
  6   -2.772170722842       -9.02       -4.41    1.0    193ms
  7   -2.772170723009       -9.78       -5.21    1.0    174ms
  8   -2.772170723015      -11.23       -5.75    2.0    200ms
  9   -2.772170723015      -12.53       -6.56    1.0    200ms
 10   -2.772170723015      -13.67       -6.80    1.0    190ms
 11   -2.772170723015   +  -14.31       -7.19    1.0    202ms
 12   -2.772170723015      -14.65       -7.37    1.0    191ms
 13   -2.772170723015   +  -14.75       -7.88    2.0    214ms
 14   -2.772170723015   +  -14.35       -8.08    1.0    192ms
n     Ene

1.7735581037663906

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770795054502                   -0.53   10.0         
  2   -2.772053188698       -2.90       -1.30    1.0    165ms
  3   -2.772082716601       -4.53       -2.71    1.0    167ms
  4   -2.772083409791       -6.16       -3.63    2.0    219ms
  5   -2.772083416546       -8.17       -4.00    1.0    170ms
  6   -2.772083417790       -8.91       -4.78    2.0    218ms
  7   -2.772083417808      -10.74       -5.34    1.0    173ms
  8   -2.772083417811      -11.54       -6.71    2.0    214ms
  9   -2.772083417811      -14.27       -7.24    1.0    191ms
 10   -2.772083417811      -13.77       -7.46    1.0    201ms
 11   -2.772083417811   +  -14.40       -8.94    1.0    182ms

Polarizability via ForwardDiff:       1.7725349724477377
Polarizability via finite difference: 1.7735581037663906
